In [1]:
import time
import numpy as np
import pickle
import sys
import os
import pybullet as p
from init import *
from utils.tools import *
from modules.RRTstar import *
import pybullet_data
from modules.simple_control import *
from modules.dwa import *
import copy

In [2]:
# connect pybullet and initialize
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
root_dir = os.path.join(os.path.dirname(os.path.realpath('file')),"../")
obstacle_ids = init_scene(p)
robotId = load_robot(p)
dy_obs = p.loadURDF("cube.urdf", [-1, -1.5, 0], globalScaling = 0.2)
p.setTimeStep(1.0 / 240.0)
p.setRealTimeSimulation(1)

In [3]:
# define joint id, range
right_front_wheel_joint = 2
right_back_wheel_joint = 3
left_front_wheel_joint = 6
left_back_wheel_joint = 7
base_wheels = [2,3,6,7]
left_wheels = [left_back_wheel_joint, left_front_wheel_joint]
right_wheels = [right_front_wheel_joint, right_back_wheel_joint]
xmin, xmax = -1.5, 3.5  # x轴范围
ymin, ymax = -4.5, 0.5  # y轴范围
obstacle_aabbs = [p.getAABB(obstacle_id) for obstacle_id in obstacle_ids]
target_body_id = None
path_node_id = []
dy_obstacle_ids = copy.deepcopy(obstacle_ids)
dy_obstacle_ids.append(dy_obs)
dy_obs_path = [(-1., 0.5), (-1., -1.5), (0.5, -1.5), (0.5, -2.5), (-1.5, -2.5), (-0.5, -4.),(0., -4.5), (-0.5, -4.),(-1.5, -2.5),(0.5, -2.5),(0.5, -1.5), (-1., -1.5)]
dwa = DynamicWindowApproach(dy_obstacle_ids)

In [4]:
def dynamic_obstacle_step(obstacle, target, speed):
    """control dynamic obstacle moving toward target"""
    pos, _ = p.getBasePositionAndOrientation(obstacle)
    relate_pos = (target[0] - pos[0], target[1] - pos[1])
    dis = np.hypot(relate_pos[0], relate_pos[1])
    if dis < 0.01:
        return True
    new_pos = [0,0,pos[2]]
    new_pos[0] = pos[0] + speed * relate_pos[0] / dis
    new_pos[1] = pos[1] + speed * relate_pos[1] / dis
    p.resetBasePositionAndOrientation(obstacle, new_pos, p.getQuaternionFromEuler([0, 0, 0]))
    return False

def dynamic_obstacle_cycle(obstacle, path, idx, speed):
    """control dynamic obstacle moving along the path"""
    reached = dynamic_obstacle_step(obstacle, path[idx], speed)
    if reached:
        return (idx + 1) % len(path)
    else:
        return idx
    
def test_obstacle_moving(dy_obs, obs_path_idx, dy_obs_path):
    p.resetBasePositionAndOrientation(dy_obs, [dy_obs_path[0][0], dy_obs_path[0][1], 0], p.getQuaternionFromEuler([0, 0, 0]))
    while True:
        obs_path_idx = dynamic_obstacle_cycle(dy_obs, dy_obs_path, obs_path_idx, 0.01)
        time.sleep(1./240)

def generate_target_visualize(target_body_id, xmin, xmax, ymin, ymax, obstacle_aabbs):
    """generate target and visualize"""
    if target_body_id:
        p.removeBody(target_body_id)
    target_position = generate_random_target(xmin, xmax, ymin, ymax, obstacle_aabbs)
    print(target_position)
    target_visual_shape_id = p.createVisualShape(shapeType=p.GEOM_SPHERE, radius=0.2, rgbaColor=[1, 0, 0, 1])
    target_body_id = p.createMultiBody(baseVisualShapeIndex=target_visual_shape_id, basePosition=target_position)
    return target_position, target_body_id


In [5]:
def test_dwa_plan(robotId, node):
    """test dwa calculate best v, w"""
    pos, orientation = p.getBasePositionAndOrientation(robotId)
    _, _, theta = p.getEulerFromQuaternion(orientation)
    v, w = dwa.plan_new(pos[0], pos[1], theta, 0, -0.1, node)
    return v, w

def test_dwa_control(robotId, path):
    """test dwa control robot to the path next node"""
    target = path[1]
    while True:
        pos, ori = p.getBasePositionAndOrientation(robotId)
        _, _, theta = p.getEulerFromQuaternion(ori)
        x, y = pos[0], pos[1]

        if np.hypot(target[0] - x, target[1] - y) < dwa.goal_thresh:
            print("Goal reached!")
            simple_control_step(robotId, 0, 0)
            break
        current_v, current_w = 0, 0
        target_v, target_w = dwa.plan(x, y, theta, current_v, current_w, target)
        simple_control_step(robotId, target_v, target_w, left_wheels=[6,7], right_wheels=[2,3])
        current_v, current_w = target_v, target_w
        time.sleep(1/240)
    simple_control_step(robotId, 0, 0)

def test_dwa_control_path_with_dyobs(obs_path_idx, path, dy_obs, dy_obs_path):
    """test dwa control robot move along the path while dynamic obstacle moving along its path"""
    next_goal_id = 0
    while next_goal_id <= len(path) - 1:
        obs_path_idx = dynamic_obstacle_cycle(dy_obs, dy_obs_path, obs_path_idx, 0.002)
        next_goal = path[next_goal_id]
        pos, ori = p.getBasePositionAndOrientation(robotId)
        _, _, theta = p.getEulerFromQuaternion(ori)
        x, y = pos[0], pos[1]

        if np.hypot(next_goal[0] - x, next_goal[1] - y) < 0.1:
            print("Goal reached!")
            next_goal_id += 1
            continue
        current_v, current_w = 0, 0
        target_v, target_w = dwa.plan(x, y, theta, current_v, current_w, next_goal)
        simple_control_step(robotId, target_v, target_w, left_wheels=[6,7], right_wheels=[2,3])
        current_v, current_w = target_v, target_w
        time.sleep(1/240)
    simple_control_step(robotId, 0, 0)
    return obs_path_idx

def test_dwa_control_path(path):
    """test dwa control robot move along the path without dynamic obstacle moving"""
    next_goal_id = 0
    while next_goal_id <= len(path) - 1:
        next_goal = path[next_goal_id]
        pos, ori = p.getBasePositionAndOrientation(robotId)
        _, _, theta = p.getEulerFromQuaternion(ori)
        x, y = pos[0], pos[1]

        if np.hypot(next_goal[0] - x, next_goal[1] - y) < 0.2:
            print("Goal reached!")
            next_goal_id += 1
            continue
        current_v, current_w = 0, 0
        target_v, target_w = dwa.plan(x, y, theta, current_v, current_w, next_goal)
        simple_control_step(robotId, target_v, target_w, left_wheels=[6,7], right_wheels=[2,3])
        current_v, current_w = target_v, target_w
        time.sleep(dwa.dt)
        
def visualize_path_node(path_node_id, path):
    """visualize path node"""
    if path_node_id:
        for id in path_node_id:
            p.removeBody(id)
    path_node_id = []
    for x in path:
        target_visual_shape_id = p.createVisualShape(shapeType=p.GEOM_SPHERE, radius=0.1, rgbaColor=[1, 0, 0, 1])
        id = p.createMultiBody(baseVisualShapeIndex=target_visual_shape_id, basePosition=[x[0], x[1], 0])
        path_node_id.append(id)
    return path_node_id


In [6]:
# test chain target with moving obstacles
obs_path_idx = 1
reset_robot(p, robotId, base_wheels)
for _ in range(5):
    target_position, target_body_id = generate_target_visualize(target_body_id, xmin, xmax, ymin, ymax, obstacle_aabbs)

    rrt_star = RRTStar(start=p.getBasePositionAndOrientation(robotId)[0][:2], goal=target_position[:2], obstacle_list=obstacle_ids, bounds=[xmin, xmax, ymin, ymax])
    path = rrt_star.plan()

    if path is not None:
        print("Global path found:", path)
        path_node_id = visualize_path_node(path_node_id, path)
        obs_path_idx = test_dwa_control_path_with_dyobs(obs_path_idx, path[1:], dy_obs, dy_obs_path)
    else:
        print("Failed to find a path")
        # p.removeBody(target_body_id)
        break
    simple_control_step(robotId, 0, 0)

[-0.8289355173873489, -0.6696045401258908, 0]
Global path found: [(-1.0, -1.0), (-0.8289355173873489, -0.6696045401258908)]
Goal reached!
[-0.35180049125555213, 0.4174234735868536, 0]
Global path found: [(-0.8994505637415139, -0.7376764080243056), (-0.35180049125555213, 0.4174234735868536)]
Goal reached!
[-1.18862013855049, 0.3830028068434155, 0]
Global path found: [(-0.39178264216737635, 0.32759300547735803), (-1.18862013855049, 0.3830028068434155)]
Goal reached!
[2.9671948143460396, -4.4866331457578905, 0]
Global path found: [(-1.10695373973732, 0.4336408479244058), (np.float64(-0.6356114501020325), np.float64(-0.7679405761988971)), (np.float64(-0.04240732911697381), np.float64(-1.4422430706621334)), (np.float64(0.5254793707787864), np.float64(-1.634994854899944)), (np.float64(0.46378641908700247), np.float64(-2.2213468131465124)), (np.float64(-0.12643807711652588), np.float64(-2.33645114195189)), (np.float64(-0.1386196767727751), np.float64(-3.9880962963389144)), (np.float64(0.45962

In [7]:
# # test chain target without moving obstacles
# # target_body_id = None
# reset_robot(p, robotId, base_wheels)
# while True:
#     target_position, target_body_id = generate_target_visualize(target_body_id)

#     rrt_star = RRTStar(start=p.getBasePositionAndOrientation(robotId)[0][:2], goal=target_position[:2], obstacle_list=obstacle_ids, bounds=[xmin, xmax, ymin, ymax])
#     path = rrt_star.plan()

#     if path is not None:
#         print("Global path found:", path)
#         test_dwa_control_path(path[1:])
#     else:
#         print("Failed to find a path")
#         # p.removeBody(target_body_id)
#         break
#     simple_control_step(robotId, 0, 0)
    